In [334]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dr="/content/drive/My Drive/Classroom/Datasets/"

In [0]:
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [0]:
para="many people told me that having son is better than having daughter . when i was five years old playing with three of my friends we found fifty five hundred dollar note and thoughts to share among us . but unfortunatel it was taken by sixty year old men who lives in u.s and he works in n a s a."

In [0]:
words=para.split()
seq_id=[len(words)+(i*0.1) for i in range(len(words))]

In [0]:
data=pd.DataFrame({'words':words,'sent_id':seq_id})

In [0]:
# loading
with open(dr+'label_spoken_tokenizer.pickle', 'rb') as handle:
    label_tokenizer = pickle.load(handle)

In [0]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines,seq_info):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    seq=seq.astype(float)
    seq.T[-1]=seq_info#adding sequence and token information
    return seq

In [0]:
testX = encode_sequences(label_tokenizer, 5, data.iloc[:,0],data.iloc[:,1].tolist())

In [0]:
label_model=load_model(dr+'lstm_label_model.h5')


In [0]:
ar=label_model.predict(testX)

In [0]:
labels=['PLAIN_PUNC','NUMERIC','LETTERS_VERB', 'ELECTRONIC' ]

In [0]:
db=[ar[e].argmax() for e in range(len(ar))]
data['new_class']=db
data['dup']=data['words'].apply(lambda w: 2 if len(w)==1 else(2 if len(w.split('.'))>1 and len(w.split('.'))<5 else 0)) 
data['new_class']=data[["new_class", "dup"]].max(axis=1)

In [0]:
n=[]
for i,w in enumerate(data['new_class']):
  if(w==0):
    c=0
    q=0
    n.append(i)
  elif(w==1):
    if(c+1==i):
      n.append(k)
      c=i
    else:
      n.append(i)
      c=i
      k=i
  elif(w==2):
    if(q+1==i):
      n.append(p)
      q=i
    else:
      n.append(i)
      q=i
      p=i
      pass
    pass
  else:
    n.append(i)
data['dup']=n

In [0]:
df=data.groupby(['dup','new_class'])['words'].apply(lambda x: ' '.join(x.astype(str))).reset_index(level =0)
df=df.reset_index()

In [0]:
#class
num_df=df[df['new_class']==1]
let_df=df[df['new_class']==2]

In [0]:
#load tokenizers
# loading
with open(dr+'letter_spoken_tokenizer.pickle', 'rb') as handle:
    letter_spoken_tokenizer = pickle.load(handle)

with open(dr+'letter_written_tokenizer.pickle', 'rb') as handle:
    letter_written_tokenizer = pickle.load(handle)

with open(dr+'number_spoken_tokenizer.pickle', 'rb') as handle:
    number_spoken_tokenizer = pickle.load(handle)
    
with open(dr+'number_written_tokenizer.pickle', 'rb') as handle:
    number_written_tokenizer = pickle.load(handle)

In [0]:
num_model=load_model(dr+'num_model.h5')
letter_model=load_model(dr+'letter_model.h5')

In [0]:
#x_num=number_spoken_tokenizer.texts_to_sequences(num_df['words'])
def encode(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [0]:
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

In [0]:
def vect_to_text(preds,tokenizer):
  preds_text = []
  for i in preds:
      temp = []
      for j in range(len(i)):
          t = get_word(i[j], tokenizer)
          if j > 0:
              if (t == get_word(i[j-1], tokenizer)) or (t == None):
                  temp.append('')
              else:
                  temp.append(t)
          else:
              if(t == None):
                  temp.append('')
              else:
                  temp.append(t) 

      preds_text.append(' '.join(temp))
  return preds_text

In [0]:
#For numerical predicts
num_x = encode(number_spoken_tokenizer, 20,num_df['words'])
preds = num_model.predict_classes(num_x.reshape((num_x.shape[0],num_x.shape[1])))

In [0]:
#leter predicts
let_x = encode(letter_spoken_tokenizer, 20,let_df['words'])
num_preds = num_model.predict_classes(let_x.reshape((let_x.shape[0],let_x.shape[1])))

In [358]:
nums=vect_to_text(preds,number_written_tokenizer)
num_df['preds']=nums


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,new_class,dup,words,preds
16,1,16,five,5
21,1,21,three,3
27,1,27,fifty five hundred,56
43,1,45,sixty,60


In [359]:
letters=vect_to_text(num_preds,letter_written_tokenizer)
let_df['preds']=letters
let_df['preds']=let_df['words'].apply(lambda w:(w.replace(" ", "") ).upper())
let_df['preds']=let_df['preds'].str.strip()#remove spaces


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,new_class,dup,words,preds
12,2,12,.,.
14,2,14,i,I
36,2,38,.,.
50,2,52,u.s,U.S
55,2,57,n a s a.,NASA.


In [366]:
final_data=data[data['new_class']==0]
final_data['preds']=final_data['words']
final_data['preds']=final_data['preds'].str.strip()#remove spaces
final_data.drop(['sent_id'],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pyd

In [0]:
frames = [final_data, let_df, num_df]

In [0]:
result = pd.concat(frames)
result=result.sort_values(by=['dup'])
result['preds']=result['preds'].str.strip()


In [0]:
paragraph=' '.join(result['preds'].values)

In [367]:
paragraph #final paragraph after conversion

'many people told me that having son is better than having daughter . when I was 5 years old playing with 3 of my friends we found 56 dollar note and thoughts to share among us . but unfortunatel it was taken by 60 year old men who lives in U.S and he works in NASA.'